In [1]:
# Establish connections to the input db and variables spreadsheets

# Import modules
import os, re, math
import arcpy
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from dotenv import load_dotenv

# Lot the .env variables
load_dotenv()

# Set the data paths
inputs_path = os.getenv('J111_INPUTS')
outputs_path = os.getenv('J111_OUTPUTS')
spatial_path = os.getenv('J111_SPATIAL')
env_path = os.getenv('J111_ENV')

# Set path to the input excel workbooks
da_xlsx = os.path.join(inputs_path, 'INPUT_DevelopmentApplications.xlsx')
proj_xlsx = os.path.join(inputs_path, 'INPUT_InterimProjections_NonResidential.xlsx')

# Get full path of Input GDBs
pscap_gdb_path = os.path.join(spatial_path, 'redland_gam_ps_capacity.gdb')
curdev_gdb_path = os.path.join(spatial_path, 'redland_gam_current_dev.gdb')
alloc_gdb_path = os.path.join(spatial_path, 'redland_gam_alloc.gdb')

# Set arcpy workspace
arcpy.env.workspace = os.path.join(env_path, 'J111_redland_gam.gdb')

# *Variables*

In [2]:
''' 
----------------------------------
Define allocation variables
----------------------------------
'''

years = [2026, 2031, 2036, 2041, 2046, 2051]
nonres_cols = [
    'commercial',
    'community',
    'education',
    'health',
    'industrial',
    'other',
    'retail',
]

# *Development Assessment Data*

In [3]:
''' 
----------------------------------
Import development assessment data
----------------------------------
'''

# Import development assessment data table
da_data = pd.read_excel(da_xlsx, 'DevelopmentApplications').drop(['app_detail', 'address', 'decision'], axis=1)

# Inspect
da_data.head()

,appid,landnos,decision_date,att_dwl,det_dwl,commercial,community,education,health,industrial,retail,other
0,MCU013891,115384;115322,43115,12,0,0,0,0,0,0,0,0
1,MCU17/0111,118178;118352;118341;118185,43159,72,0,0,0,0,0,0,0,0
2,MCU18/0037,995322,43171,14,0,0,0,0,0,0,0,0
3,MCU013865,132331,43173,0,0,0,185,0,0,0,0,720
4,MCU18/0001,314780,43214,0,0,0,0,0,0,0,0,0


In [4]:
''' 
----------------------------------
Import and prepare the parcels to council land relational dataframe
----------------------------------
'''

# Import the Council Property to DCDB Base lots relational list
parcels_to_land = pd.DataFrame.spatial.from_table(os.path.join(curdev_gdb_path, 'INPUT_TB_DCDB_CouncilPropertyRel')).drop(['OBJECTID'], axis=1)

# Select only the required column
parcels_land = parcels_to_land.loc[:, ('segpar', 'landnos')]

# Split the land numbers string into list
parcels_land.landnos = parcels_land.landnos.apply(lambda v: [int(x) for x in v.split(';')])

# Explode on the land numbers column (this will create duplicate parcel segpar records)
parcels_land = parcels_land.explode('landnos', ignore_index=True).rename({'landnos' : 'landno'}, axis=1)

# Inspect
parcels_land.head()

,segpar,landno
0,28663092,146617
1,28460061,996564
2,32043083,145846
3,20484023,144907
4,20689095,112272


In [5]:
''' 
----------------------------------
Relate DA data to land parcels
----------------------------------
'''

# Get subset of the DA data
da_land = da_data.loc[:, ('appid', 'landnos')]

# Set and explode list on the landnos
da_land['landnos'] = da_land['landnos'].astype(str) + ';'
da_land['landnos'] = da_land['landnos'].str.split(';')
da_land['landnos'] = da_land.apply(lambda row: list(filter(None, row.landnos)), axis=1)
da_land = da_land.explode('landnos').rename({'landnos' : 'landno'}, axis=1)
da_land['landno'] = da_land.apply(lambda row: int(str(row.landno)), axis=1)
da_land['landno'] = da_land['landno'].astype('Int64')

# Get the segpar for each landno
da_land = pd.merge(da_land, parcels_land, on='landno', how='left')

# Import the aggregate details table from geodatabase
agg_details = pd.DataFrame.spatial.from_table(os.path.join(curdev_gdb_path, 'GEN_TB_DCDB_AggregateDetails')).drop(['OBJECTID'], axis=1)

# Drop duplicates from the aggregate details
agg_details = agg_details.drop(['aggtype', 'segpar'], axis=1).drop_duplicates().sort_values(by='aggid').reset_index(drop=True)

# Join aggid to DA data via landno
da_land = pd.merge(da_land, agg_details, on='landno', how='left')

# Split DA data into those with an aggid and those without
da_land_agg = da_land.query("aggid == aggid").drop(['landno', 'segpar'], axis=1).drop_duplicates().reset_index(drop=True)
da_land_other = da_land.query("aggid != aggid").drop(['landno', 'aggid'], axis=1).drop_duplicates().reset_index(drop=True)

# Inspect
da_land_agg.head()

,appid,aggid
0,MCU18/0037,AGG000244
1,MCU17/0053,AGG000832
2,MCU18/0043,AGG000312
3,MCU17/0053.01,AGG000832
4,MCU18/0220,AGG002528


In [6]:
# Import propbase
propbase = pd.DataFrame.spatial.from_featureclass(os.path.join(curdev_gdb_path, 'GEN_FC_PropertyBase')).drop(['OBJECTID', 'SHAPE'], axis=1)

# Inspect
propbase.head()

,pbno,segpar,aggid,sa2_name
0,8000000,NaN,AGG000001,Cleveland
1,8000001,NaN,AGG000002,Cleveland
2,8000002,NaN,AGG000003,Redland Islands
3,8000003,NaN,AGG000004,Cleveland
4,8000004,NaN,AGG000005,Capalaba


In [7]:
''' 
----------------------------------
Relate DA data to propbase
----------------------------------
'''

# Split the propbase into aggregates and otherwise
pb_agg = propbase.query("aggid == aggid").drop(['segpar', 'sa2_name'], axis=1).drop_duplicates().reset_index(drop=True)
pb_other = propbase.query("aggid != aggid").drop(['aggid', 'sa2_name'], axis=1).drop_duplicates().reset_index(drop=True)
pb_other['segpar'] = pb_other['segpar'].astype('Int64')

# Join the propbase data onto the respective dataframes and concatenate
pb_da = pd.concat([
    pd.merge(da_land_agg, pb_agg, on='aggid', how='left').drop(['aggid'], axis=1),
    pd.merge(da_land_other, pb_other, on='segpar', how='left').drop(['segpar'], axis=1)
]).sort_values('pbno').reset_index(drop=True)

# Join DA data back onto related table
pb_da = pd.merge(pb_da, da_data.drop(['landnos'], axis=1), on='appid', how='left')

# ----------------------------------
# IMPORTANT !!!

# If there are multiple applications on a property take the most recent
pb_da = pb_da.sort_values('decision_date').drop_duplicates('pbno', keep='last').drop(['decision_date'], axis=1).reset_index(drop=True)

# ----------------------------------

# Create a subset of the table and allocate development data where situated across multiple properties
pb_da_sub = pb_da.drop(['pbno'], axis=1)
pb_da_sub['props'] = 1
ps_da_dub_g1 = pb_da_sub.loc[:, ('appid', 'props')].groupby('appid').agg('count').reset_index()
ps_da_dub_g2 = pb_da_sub.groupby('appid').agg('max').drop(['props'], axis=1).reset_index()
pb_da_sub = pd.merge(ps_da_dub_g1, ps_da_dub_g2, on='appid', how='left')

# Force all values on development columns as floats
pb_da_sub.iloc[:,2:] = pb_da_sub.iloc[:,2:].astype(float)

# Divide the development columns by the number to props to allocate
pb_da_sub.iloc[:,2:] = pb_da_sub.iloc[:,2:].div(pb_da_sub.props, axis=0)

# Drop columns
pb_da_sub = pb_da_sub.drop(['props'], axis=1)

# ----------------------------------

# Merge cleaned data back onto the probase-da table
pb_da = pd.merge(pb_da.iloc[:,:2], pb_da_sub, on='appid', how='left')

# Split the residential and non-residential components into separate dataframes
pb_da_res = pb_da.drop([col for col in pb_da.columns if col not in ['app_id', 'pbno', 'att_dwl', 'det_dwl']], axis=1)
pb_da_nonres = pb_da.drop(['det_dwl', 'att_dwl'], axis=1)

# Inspect
pb_da_res.head()

,pbno,att_dwl,det_dwl
0,8056279,6.0,0.0
1,8023850,6.0,0.0
2,8000243,14.0,0.0
3,8019617,0.0,0.0
4,8049235,0.0,0.0


# *Current Development Statistics*

In [8]:
''' 
----------------------------------
Import and prepare current development stats
----------------------------------
'''

# Import current development metrics from geodatabase
pb_curdev_lyr = pd.DataFrame.spatial.from_featureclass(os.path.join(curdev_gdb_path, 'OUT_FC_PropertyBase_CurrentDev')).drop(['OBJECTID'], axis=1)

# Drop residential columns
pb_curdev = pb_curdev_lyr.copy().drop(['SHAPE', 'prop_use', 'det_dwl', 'att_dwl', 'accom_long'], axis=1)

# Summarise current non-residential development into broader development groups

pb_curdev['commercial'] = pb_curdev['office']
pb_curdev['community'] = pb_curdev['emer_serv'] + pb_curdev['pl_of_ass']
pb_curdev['education'] = pb_curdev['childcare'] + pb_curdev['education']
pb_curdev['health'] = pb_curdev['care_accom'] + pb_curdev['hospital'] + pb_curdev['health_serv']
pb_curdev['industrial'] = pb_curdev['heavy_ind'] + pb_curdev['light_ind']
pb_curdev['other'] = pb_curdev['accom_short'] + pb_curdev['other'] + pb_curdev['rural'] + pb_curdev['indoor_ent'] + pb_curdev['indoor_rec']
pb_curdev['retail'] = pb_curdev['bulk_goods'] + pb_curdev['food_drink'] + pb_curdev['retail']

# Drop redundant non-residential columns
drop_nonres_col = [
    'office',
    'accom_short',
    'emer_serv',
    'pl_of_ass',
    'childcare',
    'care_accom',
    'hospital',
    'health_serv',
    'heavy_ind',
    'light_ind',
    'rural',
    'bulk_goods',
    'food_drink',
    'indoor_ent',
    'indoor_rec'
]
pb_curdev = pb_curdev.drop(drop_nonres_col, axis=1)

# Get total non-residential gfa by summing nonres columns
pb_curdev['nonres_gfa'] = pb_curdev.loc[:, tuple(nonres_cols)].sum(axis=1)

# Re-name columns to identify as 2021 
pb_curdev_nonres = pb_curdev.rename({col: col + '_2021' for col in pb_curdev.columns if col not in ['pbno', 'sa2_name']}, axis=1)

# Inspect
pb_curdev_nonres.head()

,pbno,sa2_name,education_2021,other_2021,retail_2021,commercial_2021,community_2021,health_2021,industrial_2021,nonres_gfa_2021
0,8000000,Cleveland,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
1,8000001,Cleveland,0.0,0.0,0.0,3404.31,0.0,0.0,0.0,3404.31
2,8000002,Redland Islands,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
3,8000003,Cleveland,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
4,8000004,Capalaba,0.0,0.0,0.0,803.46,0.0,0.0,0.0,803.46


# *Development Capacity Statistics*

In [9]:
''' 
----------------------------------
Import and prepare development capacity table
----------------------------------
'''

# Import development table from geodatabase
pb_devcap_lyr = pd.DataFrame.spatial.from_featureclass(os.path.join(pscap_gdb_path, 'OUT_FC_PropertyBase_DevCapacity')).drop(['OBJECTID'], axis=1)

# Copy the layer into a dataframe
pb_devcap = pb_devcap_lyr.copy().drop(['zone', 'property_area', 'eda', 'SHAPE'], axis=1)

# Get total non-residential gfa by summing nonres columns
pb_devcap['nonres_gfa'] = pb_devcap.loc[:, tuple(nonres_cols)].sum(axis=1)

# Re-name columns to identify as planned capacity
pb_devcap = pb_devcap.rename({col: col + '_cap' for col in pb_devcap.columns if col not in ['pbno', 'sa2_name', 'res_conv', 'nonres_conv']}, axis=1)

# Select only those columns that will convert
pb_devcap['conv'] = pb_devcap['res_conv'] + pb_devcap['nonres_conv']
pb_devcap = pb_devcap.query("conv > 0").drop(['conv'], axis=1)

# Split the residential and non-residential conversions into separate dataframes
pb_devcap_nonres = pb_devcap.query("nonres_conv > 0").drop(['res_conv', 'det_dwl_cap', 'att_dwl_cap'], axis=1).reset_index(drop=True)

# Inspect
pb_devcap_nonres.head()

,pbno,sa2_name,nonres_conv,nonres_gfa_cap,commercial_cap,community_cap,education_cap,health_cap,industrial_cap,other_cap,retail_cap
0,8000002,Redland Islands,1,672.848,0.000,0.0,0.0,0.0,0.0,672.848,0.0
1,8000004,Capalaba,1,5188.800,5188.800,0.0,0.0,0.0,0.0,0.000,0.0
2,8000006,Redland Islands,1,542.104,0.000,0.0,0.0,0.0,0.0,542.104,0.0
3,8000020,Redland Islands,1,941.984,0.000,0.0,0.0,0.0,0.0,941.984,0.0
4,8000024,Cleveland,1,10591.375,2118.275,0.0,0.0,0.0,0.0,0.000,8473.1


# *Allocation functions*

In [10]:
# Helper function for initializing spare capacity
def InitSpareCapacity(prop, dev_type, proj_year):

    dev_year = prop[f'{dev_type}{proj_year}']
    dev_cap = prop[f'{dev_type}cap']

    if dev_year and dev_cap > 0:
        if dev_cap - dev_year > 0:
            return dev_cap - dev_year
        else:
            return 0
    else:
        return dev_cap

# Helper function for setting remaining dev values in projection year (accounting for existing data)
def SetYear(prop, dev_type, proj_year, base_year):

    dev_year = prop[f'{dev_type}{proj_year}']
    dev_prev = prop[f'{dev_type}{base_year}']

    if dev_year == 0:
        return dev_prev
    else:
        return dev_year

# Helper function to set non-residential DA fa
def SetDevAssGfa(prop, dev_type, base_year):

    da_gfa = prop[f'{dev_type}da']
    base_gfa = prop[f'{dev_type}{base_year}']

    if da_gfa == 0.0:
        return 0.0
    elif da_gfa <= base_gfa:
        return base_gfa + da_gfa
    else:
        return da_gfa


# *Non-residential growth allocation process*

In [11]:
''' 
----------------------------------
Handle DA non-residential allocation in the 2026 horizon
----------------------------------
'''
# Prepare initial dataframe
pb_alloc_nonres_da = pb_curdev_nonres.set_index('pbno')

# Set new columns to 0
for col in nonres_cols:
    pb_alloc_nonres_da[f'{col}_2026'] = 0.0
    pb_alloc_nonres_da[f'{col}_cap'] = 0.0
    pb_alloc_nonres_da[f'{col}_da'] = 0.0

pb_alloc_nonres_da['nonres_conv'] = 0

# Set 2026 columns with da data where matched on pbno
pb_alloc_nonres_da.update(pb_da_nonres.rename({col: col + '_da' for col in pb_da_nonres.columns if col != 'pbno'}, axis=1).set_index('pbno'))

# Set capacity columns where matched on pbno
pb_alloc_nonres_da.update(pb_devcap_nonres.drop(['sa2_name'], axis=1).set_index('pbno'))

# Update the gfa numbers across all columns
for col in nonres_cols:
    
    # Set the GFA of 2026 to be either the DA GFA if it is greater or the DA GFA + existing GFA if it is lesser
    pb_alloc_nonres_da[f'{col}_2026'] = pb_alloc_nonres_da.apply(lambda row: SetDevAssGfa(row, f'{col}_', 2021), axis=1)

    # If there is da data for 2026 subtract this from the capacity
    pb_alloc_nonres_da[f'{col}_cap'] = pb_alloc_nonres_da.apply(lambda row: max(0.0, row[f'{col}_cap'] - row[f'{col}_2026']) if row[f'{col}_2026'] > 0 else row[f'{col}_cap'],axis=1)

# Set the projection area based on SA2
pb_alloc_nonres_da['area_name'] = pb_alloc_nonres_da.apply(lambda row: 'Balance' if row['sa2_name'] != 'Redland Islands' else row['sa2_name'], axis=1)

# Inspect
pb_alloc_nonres_da.sort_values('nonres_conv', ascending=False).head()

,sa2_name,education_2021,other_2021,retail_2021,commercial_2021,community_2021,health_2021,industrial_2021,nonres_gfa_2021,commercial_2026,...,industrial_cap,industrial_da,other_2026,other_cap,other_da,retail_2026,retail_cap,retail_da,nonres_conv,area_name
pbno,,,,,,,,,,,,,,,,,,,,,
8001327,Thorneside,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,985.938,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,1.0,Balance
8022521,Redland Islands,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,513.4160,0.0,1.0,Redland Islands
8046468,Redland Islands,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,253.4256,0.0,1.0,Redland Islands
8046469,Redland Islands,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,253.4292,0.0,1.0,Redland Islands
8046470,Redland Islands,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,509.766,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,1.0,Redland Islands


In [12]:
''' 
----------------------------------
Import QGSO non-residential projections
----------------------------------
'''

# Import detached dwelling projections
nonres_proj = pd.read_excel(proj_xlsx, 'OUT_NonResProjections')

# Select only the growth columns
nonres_proj = nonres_proj.loc[:, ~nonres_proj.columns.str.startswith('proj_')]

# Create a dictionary that holds a single dataframe for each non-residential category
nonres_projs = {}

# Loop through non-residential development categories
for category in nonres_cols:

    # Select the rows for this category
    category_df = nonres_proj.query("sector == @category").drop(['sector'], axis=1).reset_index(drop=True)

    # Rename the growth columns accordingly
    category_df = category_df.rename({col: f'{category}_' + col.lstrip('growth_') for col in category_df.columns if col != 'area_name'}, axis=1)

    # Push dataframe into dictionary
    nonres_projs[category] = category_df.set_index('area_name')

# Inspect
nonres_projs['industrial']

,industrial_2026,industrial_2031,industrial_2036,industrial_2041,industrial_2046,industrial_2051
area_name,,,,,,
Balance,59637.445708,73227.146142,60423.626725,60423.626725,60935.412415,61451.532909
Redland Islands,1427.554292,1752.853858,1446.373275,1446.373275,1458.623999,1470.978486


In [13]:
''' 
----------------------------------
Correct the non-residential growth projections to account for the already allocated DA data
----------------------------------
'''

# Summarise the DA data by projection area
da_nonres_sum = pb_alloc_nonres_da.drop(['sa2_name'], axis=1).reset_index(drop=True).loc[:, ('area_name',) + tuple([f'{col}_da' for col in nonres_cols])].groupby('area_name').agg('sum')

# Get a copy of the non-residential projections dataframes
nonres_projs_corr = nonres_projs

for cat, proj in nonres_projs_corr.items():
        
    carry = da_nonres_sum[f'{cat}_da']

    for year in years:

        sub = proj[f'{cat}_{year}'].sub(carry)
        proj[f'{cat}_{year}'] = sub.apply(lambda x: 0 if x <= 0 else x)
        
        carry = sub.apply(lambda x: abs(x) if x < 0 else 0)

# Inspect
nonres_projs_corr['industrial']

,industrial_2026,industrial_2031,industrial_2036,industrial_2041,industrial_2046,industrial_2051
area_name,,,,,,
Balance,46212.445708,73227.146142,60423.626725,60423.626725,60935.412415,61451.532909
Redland Islands,175.554292,1752.853858,1446.373275,1446.373275,1458.623999,1470.978486


In [32]:
''' 
----------------------------------
Allocate non-residential growth projections per property based on remaining capacity
----------------------------------
'''

# Prepare initial dataframe
pb_alloc_nonres = pb_alloc_nonres_da.copy()

# Drop DA GFA columns
pb_alloc_nonres = pb_alloc_nonres.drop([f'{col}_da' for col in nonres_cols], axis=1)

# Set the projection area
pb_alloc_nonres['area_name'] = pb_alloc_nonres_da.apply(lambda row: 'Balance' if row['sa2_name'] != 'Redland Islands' else row['sa2_name'], axis=1)

# Set spare capacity
for col in nonres_cols:
    pb_alloc_nonres[f'{col}_spare'] = pb_alloc_nonres.apply(lambda row: InitSpareCapacity(row, f'{col}_', 2026), axis=1)

# Set first projeciton year (2026)
for col in nonres_cols:
    pb_alloc_nonres[f'{col}_{2026}'] = pb_alloc_nonres.apply(lambda row: SetYear(row, f'{col}_', 2026, 2021), axis=1)

# Drop residential conversion column as this is now redundant
pb_alloc_nonres = pb_alloc_nonres.drop(['nonres_conv'], axis=1)

# ----------------------------------
# Set empty years

for col in nonres_cols:

    for year in years[1:]:

        pb_alloc_nonres[f'{col}_{year}'] = 0.0

# ----------------------------------
# Set column order
nonres_col_order = ['area_name', 'sa2_name']

for cat in nonres_cols:

    nonres_col_order += [f'{cat}_{year}' for year in [2021] + years] + [f'{cat}_spare', f'{cat}_cap']

pb_alloc_nonres = pb_alloc_nonres[nonres_col_order]

# ----------------------------------
# ALLOCATION LOOP

# For each of the projection categories
for cat, projdf in nonres_projs_corr.items():

    # For each projection year
    for i, year in enumerate(years):

        # Set the non-residential development column names
        dev_year = f'{cat}_{year}'
        dev_prev = f'{cat}_{year - 5}'
        dev_cap = f'{cat}_cap'
        dev_spare = f'{cat}_spare'

        # Get the summary statistics for the allocation
        year_spare = pb_alloc_nonres.reset_index().loc[:, ('area_name', dev_spare)].groupby('area_name').sum().squeeze()
        year_proj = projdf.reset_index().loc[:, ('area_name', dev_year)].set_index('area_name').squeeze()

        # Calculate the properties contribution to the spare capacity within the area based on its development capacity
        pb_alloc_nonres['contrib'] = pb_alloc_nonres.apply(lambda row: row[dev_cap] / year_spare[f'{row.area_name}'] if row[dev_spare] > 0 else 0, axis=1)

        # For each projection area grab only those properties with a contribution
        for area_name, proj in year_proj.iteritems():
            
            # Get subset of the allocation dataframe
            pb_alloc_sub = pb_alloc_nonres.query("area_name == @area_name").reset_index().loc[:, ('pbno', f'{dev_prev}', f'{dev_year}', f'{dev_spare}', 'contrib')]

            # Sort the subset from the top down
            pb_alloc_sub = pb_alloc_sub.sort_values('contrib', ascending=False)

            # Loop through the subset and allocate growth carrying over remainder when allocating
            alloc = proj

            for index, prop in pb_alloc_sub.iterrows():
                
                prop_dev_cur = prop[f'{dev_year}']
                prop_dev_prev = prop[f'{dev_prev}']
                prop_dev_spare = prop[f'{dev_spare}']

                if prop_dev_cur == 0.0 and prop_dev_spare:
                    
                    if alloc >= prop_dev_spare:
                        
                        prop_dev_frac = max(0.0, prop.contrib * proj)

                        if prop_dev_spare <= 100.0:
                            
                            prop_new_dev = prop_dev_spare

                        elif prop_dev_frac >= 100.0:

                            if prop_dev_frac > prop_dev_spare:

                                prop_new_dev = prop_dev_spare

                            else:

                                prop_new_dev = prop_dev_frac

                        else:

                            continue

                    else:

                        # Set the new development as the remaining allocation
                        prop_new_dev = alloc

                    # Update the development column in the subset
                    pb_alloc_sub.at[index, f'{dev_year}'] = prop_new_dev + prop_dev_prev

                    # Update the spare capacity column in the subset
                    pb_alloc_sub.at[index, f'{dev_spare}'] = prop_dev_spare - prop_new_dev

                    # Remove new development from the overall allocation
                    alloc -= prop_new_dev

                if alloc <= 0:

                    break

            # Drop extraneous columns from the subset
            pb_alloc_sub = pb_alloc_sub.drop(['contrib'], axis=1).set_index('pbno')

            # Update the master dataframe with the updated columns
            pb_alloc_nonres.update(pb_alloc_sub)

        # Set remaining null values in projection year to be that of year before (for 2031 onwards)
        if year >= 2031:
            pb_alloc_nonres[f'{dev_year}'] = pb_alloc_nonres.apply(lambda row: SetYear(row, f'{cat}_', year, years[i - 1]), axis=1)

# ----------------------------------
# Drop extraneous
pb_alloc_nonres = pb_alloc_nonres.drop(['area_name', 'contrib'], axis=1)

# Inspect
insp = 'industrial'
insp_cols = ['sa2_name'] + [f'{insp}_{year}' for year in [2021] + years] + [f'{insp}_spare', f'{insp}_cap']
pb_alloc_nonres[insp_cols].sort_values(f'{insp}_spare', ascending=False).head()

,sa2_name,industrial_2021,industrial_2026,industrial_2031,industrial_2036,industrial_2041,industrial_2046,industrial_2051,industrial_spare,industrial_cap
pbno,,,,,,,,,,
8051799,Redland Islands,0.000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,20978.310000,20978.310
8052342,Redland Islands,0.000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,18780.510000,18780.510
8006092,Redland Islands,0.000,175.554292,1928.40815,3374.781426,4821.154701,6279.7787,7750.757186,13763.262814,21514.020
8014319,Redland Islands,2183.972,2183.972000,2183.97200,2183.972000,2183.972000,2183.9720,2183.972000,12446.946000,12446.946
8001822,Redland Islands,0.000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,10954.896000,10954.896


In [33]:
''' 
----------------------------------
Get the non-residential growth summary by SA2
----------------------------------
'''
# Groupby and sum
sa2_nonres_alloc = pb_alloc_nonres.reset_index(drop=True).groupby('sa2_name').agg('sum')

# Inspect
sa2_nonres_alloc

,commercial_2021,commercial_2026,commercial_2031,commercial_2036,commercial_2041,commercial_2046,commercial_2051,commercial_spare,commercial_cap,community_2021,...,other_cap,retail_2021,retail_2026,retail_2031,retail_2036,retail_2041,retail_2046,retail_2051,retail_spare,retail_cap
sa2_name,,,,,,,,,,,,,,,,,,,,,
Alexandra Hills,261.2190,261.219000,261.219000,261.219000,261.219000,261.219000,261.219000,1925.728400,1925.7284,1345.813,...,0.00,29413.237,29413.237000,29413.237000,29413.237000,29413.237000,29413.237000,29413.237000,7702.913600,7702.9136
Birkdale,5080.6070,5080.607000,5441.973748,5701.070091,6032.558145,6427.493794,6427.493794,5705.387606,7052.2744,3658.536,...,0.00,37408.602,37408.602000,37408.602000,37408.602000,37408.602000,37408.602000,37408.602000,28209.097600,28209.0976
Capalaba,96153.7045,128454.115419,149845.216696,174260.381988,201090.134877,225949.719991,250552.492745,76553.971155,230952.7594,11628.799,...,0.00,337039.171,345792.771250,409706.247129,464637.689812,516822.560361,569475.389175,622600.372585,231262.282015,515990.7806
Cleveland,73226.2195,73226.219500,75903.378708,77436.261049,78042.308074,78764.353929,79652.859326,25883.011774,32309.6516,27317.640,...,0.00,107574.768,116780.420560,116780.420560,116780.420560,116780.420560,116780.420560,116780.420560,117843.569440,123292.0520
Ormiston,18792.3190,18792.319000,21156.867659,23556.269825,26626.066741,30283.430031,34783.957240,11178.245760,27169.8840,9847.159,...,0.00,35761.579,36008.880500,36008.880500,36008.880500,36008.880500,36008.880500,36008.880500,22255.234500,22502.5360
Redland Bay,7927.6400,7927.640000,10575.253902,12008.482110,13527.841255,15338.009326,17565.491261,13077.221139,22715.0724,5834.717,...,0.00,48667.381,60436.387900,60436.387900,60436.387900,60436.387900,60436.387900,60436.387900,79224.335500,87233.3424
Redland Islands,6832.9800,8032.569081,9346.745776,10680.617217,12197.112668,13746.427856,15329.273061,20863.220539,29359.5136,19334.021,...,20010.22,21859.614,24315.114459,26698.638581,28747.195898,30693.325348,32656.906359,34638.095422,104854.572978,117393.0544
Sheldon - Mount Cotton,0.0000,0.000000,329.475577,700.582786,1175.377624,1741.049315,1741.049315,1591.125085,3332.1744,1566.684,...,0.00,2978.906,2978.906000,2978.906000,2978.906000,2978.906000,2978.906000,2978.906000,13328.697600,13328.6976
Thorneside,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1860.830000,1860.8300,690.333,...,0.00,2699.368,2865.944295,2865.944295,2865.944295,2865.944295,2865.944295,2865.944295,7276.743705,7443.3200


# *Outputs*

In [34]:
# Output SA2 metrics to csv
sa2_nonres_alloc.to_csv(os.path.join(outputs_path, 'OUT_Redland_SA2_NonResGrowthAllocation.csv'))

In [35]:
# Merge with spatial
pb_alloc_nonres_out = pd.merge(pb_curdev_lyr.loc[:, ('pbno', 'SHAPE')], pb_alloc_nonres.reset_index(), on='pbno', how='left')

# Output into geodatabase
pb_alloc_nonres_out.spatial.to_featureclass(os.path.join(alloc_gdb_path, 'OUT_FC_PropertyBase_NonResGrowthAllocation'), overwrite=True)

'G:\\Shared drives\\PIESolutions_03_Projects\\J000111 - Redlands planning assumption update\\06_Working Documents\\00_GIS Directory\\00_Data\\gam_model_run\\redland_gam_alloc.gdb\\OUT_FC_PropertyBase_NonResGrowthAllocation'